## Weather API code form Open Meteo
https://open-meteo.com/en/docs/historical-weather-api/

## READ ME
To have the machine learning model make predictions using weather it needs to have been trained with the weather data as one of the inputs. The weather api below gets the weather history bewtween a range of dates and sends it to a dataframe.

In [ ]:
!pip install openmeteo-requests requests-cache retry-requests numpy pandas
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.3 MB/s eta 0:00:00


# READ ME
This is not the final version of the training data, so you cannot rely on copying and pasting the first and last date from it or that the dates wont change

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


youll need to change the first part of this file path

In [ ]:
csv_files = ['/content/drive/MyDrive/50_ResidentialPowerDisaggregation_SD_Fall23/1.2 Software/Colab Notebooks/Weather API/Copy of MLData0.csv']  # YOU WILL NEED TO CHANGE THE PATH TO THE FILE FOR YOUR DRIVE

# Load and concatenate data from multiple CSV files if needed
data_list = []
for csv_file in csv_files:
    data = pd.read_csv(csv_file)
    data_list.append(data)

# Concatenate data from all CSV files
training_data = pd.concat(data_list, ignore_index=True)

In [ ]:
training_data

,timestamp,Total,Washer,BlowerGH,Lights,BlowerBed,CompGH,CompBed,Dryer,Recs1,Recs2,WaterHeater
0,1/18/2023 22:00,164.1,0.5,4.6,25.7,1.7,21.7,5.8,0.0,7.1,5.6,0.0
1,1/18/2023 22:01,164.3,0.5,4.6,25.6,1.7,21.7,5.8,0.0,7.1,5.6,0.0
2,1/18/2023 22:02,164.1,0.6,4.6,25.8,1.8,21.7,5.8,0.0,7.0,5.5,0.0
3,1/18/2023 22:03,164.0,0.5,4.5,25.6,1.8,21.6,5.8,0.0,7.1,5.6,0.0
4,1/18/2023 22:04,164.2,0.6,4.6,25.7,1.7,21.7,5.8,0.0,7.0,5.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5771,1/22/2023 22:11,7091.1,6.4,8.9,41.7,225.5,25.2,629.4,970.1,10.1,7.3,0.0
5772,1/22/2023 22:12,1931.8,6.0,6.3,43.4,226.8,21.1,628.1,-0.1,7.3,5.8,0.0
5773,1/22/2023 22:13,1943.2,6.0,6.4,43.5,227.9,21.2,631.0,-0.1,7.3,5.8,0.0
5774,1/22/2023 22:14,7393.9,6.4,9.2,41.9,225.9,25.6,628.0,1026.7,10.3,7.5,0.0


# READ ME
Above is an example of what the training data will look like. The only relevant column for adding the weather history is the timestamp. One way of adding the weather history would be to do the following:


*   start date in the code below would become a varaible with the earliest timestamp from the training data
*   end date in the code below would become a variable with the lastest date from the training data
*   (This needs to be done with code instead of typing out what the timestamp is. With more training data the first and last timestamp could be different)  
*   The weather API will create a dataframe with two columns, the timestamp, and the temperature, however it is only updated every hour so the dataframe sizes aren't the same
*   You can add the weather column to the training data, but since it is only from every hour, minutes 1-60 would have the same temperature value
*   You can compare the first part of the timestamp in the training data to the first part of the timestamp for the weather data, and if they are the same you add that temperature value to the training data


Example:
Training data has timestamps 1/18/23 22:00, 1/18/23 22:01, 1/18/23 22:02, 1/18/23 22:03,... up to 1/18/23 22:59 for that hour. The weather dataframe would just have 1/18/23 22:00, 1/18/23 23:00, etc. and the temperature. By only comparing the first part of it "1/18/23 22:" and exculding the minute value you could add the weather value. copying and pasting the weather values into the spreadsheet or adding them 60 times may be easier but wouldn't work with new training data or if the data was missing a certain timestamp.








In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 35.76,
	"longitude": -79.02,
	"start_date": "2023-01-18",     #THIS IS THE START DATE, CAN REPLACE THIS WITH A DYNAMIC WAY TO GET THE EARLIEST DATE FROM THE TRAINING DATA
	"end_date": "2023-01-22",				 #THIS IS THE END DATE, CAN REPLACE THIS WITH A DYNAMIC WAY TO GET THE LATEST DATE FROM THE TRAINING DATA
	"hourly": "temperature_2m",
	"temperature_unit": "fahrenheit",
	"wind_speed_unit": "mph",
	"precipitation_unit": "inch",
	"timeformat": "unixtime",
	"timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 35.74692153930664°E -78.98196411132812°N
Elevation 64.0 m asl
Timezone b'America/New_York' b'EST'
Timezone difference to GMT+0 -18000 s
                   date  temperature_2m
0   2023-01-18 05:00:00       45.647598
1   2023-01-18 06:00:00       44.477600
2   2023-01-18 07:00:00       42.767601
3   2023-01-18 08:00:00       41.687599
4   2023-01-18 09:00:00       40.517601
..                  ...             ...
115 2023-01-23 00:00:00       42.677601
116 2023-01-23 01:00:00       42.587601
117 2023-01-23 02:00:00       42.047600
118 2023-01-23 03:00:00       41.957600
119 2023-01-23 04:00:00       41.597603

[120 rows x 2 columns]


In [ ]:
hourly_dataframe['date'] = pd.to_datetime(hourly_dataframe['data'])
hourly_dataframe

NameError: name 'pd' is not defined